# MOD 5 FINAL PROJECT - WIP
- James M. Irving, Ph.D.
- 06/20/19

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MOD-5-FINAL-PROJECT---WIP" data-toc-modified-id="MOD-5-FINAL-PROJECT---WIP-1">MOD 5 FINAL PROJECT - WIP</a></span><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1.1">Plan</a></span></li></ul></li></ul></div>

## Plan

### Goal
- Goal is to use President Trump's tweets to predict fluctuations in the stock market.

### Data to Analyze
- All Donal Trump tweets from inaugaration day 2017 to today (for now) - 06/20/19
    - Extracted from http://www.trumptwitterarchive.com/archive
    - Copy and pasted .csv into "../trump_tweets_01202017_06202019.csv"

In [14]:
import pandas as pd
file = '../trump_tweets_01202017_06202019.csv'
df = pd.read_csv(file, encoding='utf-8')

In [15]:
df.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,Since Election Day 2016 Stocks up almost 50% S...,06-20-2019 00:12:31,16742,64362,False,1141499029727121408
1,Twitter for iPhone,Congratulations to President Lopez Obrador — M...,06-19-2019 23:01:59,20490,87423,False,1141481280653209600
2,Twitter for iPhone,Just returning from Orlando and Doral (Miami) ...,06-19-2019 20:39:28,12693,68118,False,1141445414824136704
3,Twitter for iPhone,I will be interviewed LIVE tonight by @seanhan...,06-19-2019 20:29:24,10586,48786,False,1141442879518322689
4,Twitter for iPhone,....asking Hillary Clinton why she deleted and...,06-19-2019 19:48:04,21712,93137,False,1141432478491717632


In [16]:
from pandas_profiling import ProfileReport

In [17]:
ProfileReport(df)

Number of variables,7
Number of observations,8442
Total Missing (%),0.1%
Total size in memory,461.8 KiB
Average record size in memory,56.0 B
Numeric,3
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [18]:
df['date']=pd.to_datetime(df['created_at'])
df.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,date
0,Twitter for iPhone,Since Election Day 2016 Stocks up almost 50% S...,06-20-2019 00:12:31,16742,64362,False,1141499029727121408,2019-06-20 00:12:31
1,Twitter for iPhone,Congratulations to President Lopez Obrador — M...,06-19-2019 23:01:59,20490,87423,False,1141481280653209600,2019-06-19 23:01:59
2,Twitter for iPhone,Just returning from Orlando and Doral (Miami) ...,06-19-2019 20:39:28,12693,68118,False,1141445414824136704,2019-06-19 20:39:28
3,Twitter for iPhone,I will be interviewed LIVE tonight by @seanhan...,06-19-2019 20:29:24,10586,48786,False,1141442879518322689,2019-06-19 20:29:24
4,Twitter for iPhone,....asking Hillary Clinton why she deleted and...,06-19-2019 19:48:04,21712,93137,False,1141432478491717632,2019-06-19 19:48:04


In [20]:

print(df.date.max(), df.date.min())

2019-06-20 00:12:31 2017-01-20 12:31:53


In [21]:
df.date.max() - df.date.min()

Timedelta('880 days 11:40:38')

### Calculating "CAPTSratio" 
- \# of uppercase chars/ # of lowercase chars

In [48]:
test = df['text'][3]
print(test)

test_upper = [1 for x in test if x.isupper()]
test_lower = [1 for x in test if x.islower()]
test_ratio = sum(test_upper) /sum(test_lower)
print(test_ratio)

I will be interviewed LIVE tonight by @seanhannity on @FoxNews  9 P.M. Enjoy!
0.20833333333333334


In [45]:
def case_ratio(msg):
    msg_length = len(msg)
    test_upper = [1 for x in msg if x.isupper()]
    test_lower = [1 for x in msg if x.islower()]
    test_ratio = sum(test_upper)/msg_length
    
    case_dict = {'num_upper':sum(test_upper),'num_lower':sum(test_lower), 'upper_lower_ratio':test_ratio}
    return test_ratio

In [47]:
df['case_ratio'] = df['text'].apply(lambda x: case_ratio(x))
df.sort_values('case_ratio',ascending=False).head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,date,case_ratio
175,Twitter for iPhone,PRESIDENTIAL HARASSMENT!,06-11-2019 11:51:43,22053,96769,False,1138413497858101249,2019-06-11 11:51:43,0.916667
519,Twitter for iPhone,PRESIDENTIAL HARASSMENT!,05-22-2019 10:03:51,25473,108688,False,1131138596109463553,2019-05-22 10:03:51,0.916667
1788,Twitter for iPhone,PRESIDENTIAL HARASSMENT!,03-08-2019 12:34:24,18854,84483,False,1103997393073254400,2019-03-08 12:34:24,0.916667
1816,Twitter for iPhone,PRESIDENTIAL HARASSMENT!,03-05-2019 14:12:28,27152,115800,False,1102934910900363266,2019-03-05 14:12:28,0.916667
1255,Twitter for iPhone,PRESIDENTIAL HARASSMENT!,04-18-2019 12:07:08,30697,145842,False,1118848431932493824,2019-04-18 12:07:08,0.916667
